In [1]:
import tensorflow as tf
import numpy as np
from helpers.layers import Material, ConstLayer, VariableLayer
from helpers.tf_reflectance import tf_reflectance, reflectance, reflectance_gradient
from helpers.interpolate_extinction_coeffs import extinction_coeffs

2023-03-30 19:58:08.430510: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 19:58:08.559554: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-30 19:58:08.559574: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-30 19:58:09.332590: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
spectra_3cm = np.load('./data/3cm.npy').flatten()
spectra_1cm = np.load('./data/1cm.npy').flatten()
reference = np.load('./data/ref.npy').flatten()
wls, coeffs = extinction_coeffs()

coeffs = tf.constant(coeffs.T, dtype=tf.float32)
wls = tf.constant(wls[:, None], dtype=tf.float32)
obs1 = tf.constant(spectra_1cm/reference, dtype=tf.double)
obs2 = tf.constant(spectra_3cm/reference, dtype=tf.double)

2023-03-30 19:58:15.913147: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-30 19:58:15.913172: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-30 19:58:15.913193: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2023-03-30 19:58:15.913465: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
cm3_scale = tf.Variable(10.0)
cm1_scale = tf.Variable(10.0)

mat = Material(
    (VariableLayer(wls, coeffs),
    ConstLayer(0.01, 2.4),
    VariableLayer(wls, coeffs))
)

In [16]:
dist_one = tf.Variable(1.0, dtype=tf.double)
dist_two = tf.Variable(3.0, dtype=tf.double)

def loss(pred1, pred2):
    sum = tf.reduce_mean((obs1 - pred1*dist_one)**2)
    sum += tf.reduce_mean((obs2 - pred2*dist_two)**2)
    return sum

with tf.GradientTape() as t:
    res1 = mat(dist_one)
    res2 = mat(dist_two)
    loss_result = loss(res1, res2)

trainable = mat.trainable_variables + [dist_one, dist_two]

t.gradient(loss_result, trainable)

SystemError: <built-in function len> returned a result with an exception set